<a href="https://www.kaggle.com/code/hatemamine/preprocessearabicmmarco-trainset-5m-validationset?scriptVersionId=222417305" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [2]:
queries_data ='queries-arabic-preprocess-for-araelectra'
collection_data='collection-arabic-preprocess-for-araelectra'
path_dev_data = 'dev_samples_araelectra.pkl'
train_data_output="KD_train_dataset_t05M_t15m_eval_10k_preprocessedAraELECTRA"
organization_dataset_id="hatemestinbejaia/ExperimentDATA_knowledge_distillation_vs_fine_tuning"
# lib to free memory
import gc
import ctypes
libc = ctypes.CDLL("libc.so.6")

In [3]:
!pip install -U datasets huggingface-hub -q

https://zenodo.org/records/4068216
Teacher train files (ids only), based on the MSMARCO-Passage collection, for the paper: 
Improving Efficient Neural Ranking Models with Cross-Architecture Knowledge Distillation Sebastian Hofstätter, Sophia Althammer, Michael Schröder, Mete Sertkan and Allan Hanbury (https://arxiv.org/abs/2010.02666)
for the documentation please see: https://github.com/sebastian-hofstaetter/neural-ranking-kd

In [4]:
#train files (ids only), ,
# code used to download train files (ids only) based on the MSMARCO-Passage collection from https://zenodo.org/records/4068216 
# and push it to our haggingface reposit
"""
import huggingface_hub 
hf = huggingface_hub.HfFolder()
access_token = "your access token"
hf.save_token(access_token)
organization_dataset_id="your huggingface reposit"
!wget https://zenodo.org/records/4068216/files/bert_cat_ensemble_msmarcopassage_train_scores_ids.tsv
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj="bert_cat_ensemble_msmarcopassage_train_scores_ids.tsv",
    path_in_repo="bert_cat_ensemble_msmarcopassage_train_scores_ids.tsv",
    repo_id=organization_dataset_id,
    repo_type="dataset",
)
"""

'\nimport huggingface_hub \nhf = huggingface_hub.HfFolder()\naccess_token = "your access token"\nhf.save_token(access_token)\norganization_dataset_id="your huggingface reposit"\n!wget https://zenodo.org/records/4068216/files/bert_cat_ensemble_msmarcopassage_train_scores_ids.tsv\nfrom huggingface_hub import HfApi\napi = HfApi()\napi.upload_file(\n    path_or_fileobj="bert_cat_ensemble_msmarcopassage_train_scores_ids.tsv",\n    path_in_repo="bert_cat_ensemble_msmarcopassage_train_scores_ids.tsv",\n    repo_id=organization_dataset_id,\n    repo_type="dataset",\n)\n'

In [5]:
#get the train files (ids only)
from huggingface_hub import hf_hub_download
hf_hub_download(repo_id=organization_dataset_id, filename="bert_cat_ensemble_msmarcopassage_train_scores_ids.tsv", local_dir="./", repo_type="dataset")

(…)mble_msmarcopassage_train_scores_ids.tsv:   0%|          | 0.00/2.39G [00:00<?, ?B/s]

'bert_cat_ensemble_msmarcopassage_train_scores_ids.tsv'

In [6]:
import pandas as pd
df = pd.read_csv("bert_cat_ensemble_msmarcopassage_train_scores_ids.tsv",sep="\t", header=None)
df.rename(columns={0: 'pos_score', 1: 'neg_score', 2: 'query', 3: 'pos', 4: 'neg'}, inplace=True)
#we remove pos_score, neg_score as we do not needed theme for this exprience, but we will use theme in another exprience
#df.drop(columns=['pos_score', 'neg_score'], inplace=True)
#df['label'] = (df['pos_score'] - df['neg_score']).abs()
df['label'] = df['pos_score'] - df['neg_score']
df.head()

,pos_score,neg_score,query,pos,neg,label
0,6.624662,2.596603,400296,1540783,3518497,4.028059
1,7.798287,-2.383167,662731,193249,2975302,10.181454
2,5.206316,-11.214259,238256,4435042,100008,16.420576
3,6.370231,-0.851265,527862,1505983,2975302,7.221496
4,7.016524,-5.532790,275813,5736515,1238670,12.549315


In [7]:
# get a shuffle train files (ids only) to be used to generate dev_samples for evalute with MRR@10
df_shuffle = df.sample(frac=1, random_state=7).reset_index(drop=True)
df_shuffle.head()

,pos_score,neg_score,query,pos,neg,label
0,6.686563,-0.189442,229334,3145735,1141677,6.876004
1,10.221210,-9.819460,803399,4306689,2884126,20.040670
2,7.514020,-2.692827,516914,1554837,27199,10.206848
3,7.594009,-11.738208,562546,5276014,2485273,19.332217
4,7.262992,1.171521,518829,1076341,5327398,6.091472


In [8]:
#get the preprocessed queries 
from datasets import load_dataset, Dataset, load_from_disk
datasetQ = load_dataset(organization_dataset_id, queries_data, trust_remote_code=True)
print(datasetQ['train'][1])
import tqdm
queries = {}
for i in tqdm.trange(len(datasetQ['train'])):
    queries[datasetQ['train'][i]["id"]]= datasetQ['train'][i]["text"]
    #break
del datasetQ
gc.collect()
libc.malloc_trim(0)

README.md:   0%|          | 0.00/27.2k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/26.5M [00:00<?, ?B/s]

dev.full-00000-of-00001.parquet:   0%|          | 0.00/3.63M [00:00<?, ?B/s]

dev-00000-of-00001.parquet:   0%|          | 0.00/248k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/808731 [00:00<?, ? examples/s]

Generating dev.full split:   0%|          | 0/101093 [00:00<?, ? examples/s]

Generating dev split:   0%|          | 0/6980 [00:00<?, ? examples/s]

{'id': 3, 'text': 'اسم آخر للقشرة البصرية الأولية هو'}


100%|██████████| 808731/808731 [01:02<00:00, 13021.33it/s]


1

In [9]:
# get the prepocessed collection 
datasetC = load_dataset(organization_dataset_id, collection_data, trust_remote_code=True)
print(datasetC['collection'][100])
import tqdm
docs = {}
for i in tqdm.trange(len(datasetC['collection'])):
    docs[datasetC['collection'][i]["id"]]= datasetC['collection'][i]["text"]
    #break
del datasetC
gc.collect()
libc.malloc_trim(0)

collection-00000-of-00010.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

collection-00001-of-00010.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

collection-00002-of-00010.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

collection-00003-of-00010.parquet:   0%|          | 0.00/210M [00:00<?, ?B/s]

collection-00004-of-00010.parquet:   0%|          | 0.00/211M [00:00<?, ?B/s]

...

collection-00009-of-00010.parquet:   0%|          | 0.00/208M [00:00<?, ?B/s]

Generating collection split:   0%|          | 0/8841823 [00:00<?, ? examples/s]

{'id': 100, 'text': 'أنتونون دفوروك ( 1841 - 1904 ) كان أنتونين دفوراك ابن جزار ، لكنه لم يتبع تجارة والده . أثناء مساعدة والده بدوام جزئي ، درس الموسيقى وتخرج من مدرسة براغ أورغن في عام 1859 .'}


100%|██████████| 8841823/8841823 [13:07<00:00, 11225.09it/s]


1

In [10]:
### Now we create our dev data to test our models in reranking task using MRR@10 metric
dev_samples = {}
# We use 300 random queries from the train set for evaluation during training
# Each query has at least one relevant and up to 300 irrelevant (negative) passages
num_dev_queries = 300
num_max_dev_negatives = 300
for i in range(len(df_shuffle)):
    qid=df_shuffle["query"][i]
    pos_id=df_shuffle["pos"][i]
    neg_id=df_shuffle["neg"][i]
    if qid not in dev_samples and len(dev_samples) < num_dev_queries:
            dev_samples[qid] = {"query": queries[qid], "positive": set(), "negative": set()}

    if qid in dev_samples:
            dev_samples[qid]["positive"].add(docs[pos_id])

            if len(dev_samples[qid]["negative"]) < num_max_dev_negatives:
                dev_samples[qid]["negative"].add(docs[neg_id])

    #break
dev_qids = set(dev_samples.keys())

In [11]:
# to push to your huggingface reposit uncomment
"""
import pickle 
with open(path_dev_data, 'wb') as f:
    pickle.dump(dev_samples, f)
# push the dev data to our huggingface repository
from huggingface_hub import HfApi
api = HfApi()
api.upload_file(
    path_or_fileobj=path_dev_data,
    path_in_repo=path_dev_data,
    repo_id=organization_dataset_id,
    repo_type="dataset",
)
"""

'\nimport pickle \nwith open(path_dev_data, \'wb\') as f:\n    pickle.dump(dev_samples, f)\n# push the dev data to our huggingface repository\nfrom huggingface_hub import HfApi\napi = HfApi()\napi.upload_file(\n    path_or_fileobj=path_dev_data,\n    path_in_repo=path_dev_data,\n    repo_id=organization_dataset_id,\n    repo_type="dataset",\n)\n'

In [ ]:
print(df.head())
# we remove the queries selected for the dev data from the training data, to get a fair judgment
print(len(df))
df=df[~df["query"].isin(dev_qids)]
df=df.reset_index(drop=True)
print(len(df))

In [13]:
# creat our dataset used for our expriment
# first the model will be train in the train0
# than the model will be used to select Hard Negative sample from Train 1
# finaly the model will be trained with the Hard Negative sample
from datasets import DatasetDict
train_dataset0 = Dataset.from_pandas(df[:5000000])
train_dataset1 = Dataset.from_pandas(df[5000000:10000000])
train_dataset2 = Dataset.from_pandas(df[10000000:15000000])
eval_dataset = Dataset.from_pandas(df[len(df)-10000:len(df)])
dataset = DatasetDict({
    'train0': train_dataset0,
    'train1': train_dataset1,
    'train2': train_dataset2,
    'test': eval_dataset
})
dataset.save_to_disk("tmp")
dataset= load_from_disk("tmp")
dataset

Saving the dataset (0/1 shards):   0%|          | 0/5000000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/5000000 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/10000 [00:00<?, ? examples/s]

DatasetDict({
    train0: Dataset({
        features: ['pos_score', 'neg_score', 'query', 'pos', 'neg', 'label'],
        num_rows: 5000000
    })
    train1: Dataset({
        features: ['pos_score', 'neg_score', 'query', 'pos', 'neg', 'label'],
        num_rows: 5000000
    })
    train2: Dataset({
        features: ['pos_score', 'neg_score', 'query', 'pos', 'neg', 'label'],
        num_rows: 5000000
    })
    test: Dataset({
        features: ['pos_score', 'neg_score', 'query', 'pos', 'neg', 'label'],
        num_rows: 10000
    })
})

In [14]:
#free memory
del df, df_shuffle, train_dataset0, train_dataset1, eval_dataset
gc.collect()
libc.malloc_trim(0)

1

In [15]:
def ids_to_texts(batch):
    for i in range(len(batch["query"])):
        batch["query"][i] = queries[batch["query"][i]]
        batch["pos"][i] = docs[batch["pos"][i]]
        batch["neg"][i] =  docs[batch["neg"][i]]
    return batch

In [ ]:
dataset= dataset.map(ids_to_texts, batched=True, keep_in_memory=False)

Map:   0%|          | 0/5000000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000000 [00:00<?, ? examples/s]

In [ ]:
# uncomment to push to your huggingface reposit
"""
import huggingface_hub 
hf = huggingface_hub.HfFolder()
access_token = "your access token"
hf.save_token(access_token)
organization_dataset_id="your huggingface reposit"
dataset.push_to_hub(organization_dataset_id, train_data_output)
"""